In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

### 프로젝트 셋팅

In [2]:
# 학습이 완료된 모델을 저장할 파일 이름
best_model_path = 'open/best_model_classification.dat'
# 예측된 결과를 저장할 파일 이름
prediction_path = 'open/predict_classification.csv'
# 사용할 학습 모델들의 이름
model_name_list = ['KNN', 'LogisticRegression', 'SVC', 'DecisionTree', 'RandomForest',
                   'AdaBoost', 'GradientBoost', 'LGBM', 'XGBoost', 'Voting(Hard)',
                   'Voting(Soft)']
# 사용하고 싶지 않은 학습 모델 이름
# model_name_list에 있는 이름으로 넣어주세요
except_model_name_list = []

In [3]:
# 학습이 완료된 모델 객체를 담을 리스트
complete_model_list = []
# 학습이 완료된 모델 객체의 이름
complete_model_name_list = []
# 교차검증한 평가 수치를 담을 리스트
# 여기서는 f1만 하지만 다른것도 하고 싶다면 각각 리스트를 만들어주세요
f1_score_list = []
f1_score_tuning_list = []
# 교차검증 횟수
cv_count = 10

### 데이터 준비
- 데이터를 읽어오고 필요한 전처리까지 다 한다음 입력데이터는 train_X, 결과데이터는 train_y라는 변수에 담아서 준비해주세요

In [4]:
# 데이터 준비
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test.csv')
# 입력과 결과로 나눈다.
X = train.drop(['ID','Cancer'], axis=1)
y = train['Cancer']
test = test.drop('ID', axis = 1)

FileNotFoundError: [Errno 2] No such file or directory: 'open/train.csv'

In [ ]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Gender','Country','Race','Family_Background',
            'Radiation_History','Iodine_Deficiency','Smoke',
            'Weight_Risk','Diabetes']

encoders = {}                       # 나중에 저장해 두면 예측 시 복원 가능
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    test[col] = le.transform(test[col])
    encoders[col] = le              # 필요하면 pickle로 저장


In [ ]:
# 입력 데이터 표준화
scaler1 = StandardScaler()
scaler1.fit(X)
X2 = scaler1.transform(X)

In [ ]:
# 학습할 데이터를 변수에 담아준다.
train_X = X2
train_y = y

### 기본 모델 사용하기
- 기본 모델 중에 만족하는 것을 찾았다면 하이퍼 파라미터 튜닝 과정은 생략하세요

In [ ]:
# 학습모델을 만들어서 반환하는 함수
def makeModel(model_name) :
    # 모델 객체를 생성한다.
    # 다른 모델들을 추가 하셔도 됩니다.
    if model_name == 'KNN' :
        new_model = KNeighborsClassifier()
    elif model_name == 'LogisticRegression' :
        new_model = LogisticRegression()
    elif model_name == 'SVC' :
        new_model = SVC(probability=True)
    elif model_name == 'DecisionTree' :
        new_model = DecisionTreeClassifier()
    elif model_name == 'RandomForest' :
        new_model = RandomForestClassifier()
    elif model_name == 'AdaBoost' :
        new_model = AdaBoostClassifier()
    elif model_name == 'GradientBoost' :
        new_model = GradientBoostingClassifier()
    elif model_name == 'LGBM' :
        new_model = LGBMClassifier(verbose = -1)
    elif model_name == 'XGBoost' :
        new_model = XGBClassifier(silent=True, verbose = -1)
    else :
        new_model = None

    return new_model

In [ ]:
new_model_list = []
new_model_name_list = []

# 사용하지 않을 모델은 제외한다.
for m1 in model_name_list :
    # 사용하지 않을 모델이라면 다음 반복으로 넘어간다.
    if m1 in except_model_name_list :
        continue
    # Voting은 제외한다.
    if m1 in ['Voting(Hard)', 'Voting(Soft)'] :
        continue

    # 모델 객체를 생성한다.
    new_model = makeModel(m1)

    # 생성한 모델을 리스트에 담아준다.
    new_model_list.append(new_model)
    new_model_name_list.append(m1)

display(new_model_list)
display(new_model_name_list)

[KNeighborsClassifier(),
 LogisticRegression(),
 SVC(probability=True),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 AdaBoostClassifier(),
 GradientBoostingClassifier(),
 LGBMClassifier(verbose=-1),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,
               n_jobs=None, num_parallel_tree=None, ...)]

['KNN',
 'LogisticRegression',
 'SVC',
 'DecisionTree',
 'RandomForest',
 'AdaBoost',
 'GradientBoost',
 'LGBM',
 'XGBoost']

In [ ]:
# HardVoting 구성
# 사용할 모델을 담을 리스트
hard_voting_model_list = []

# 사용하고자 하는 모델들의 이름으로 반복한다.
for m1 in new_model_name_list :
    # 모델을 생성한다.
    new_model = makeModel(m1)
    # 모델을 이름과 함께 튜플로 구성한다.
    t1 = (m1, new_model)
    # 모델 리스트에 담아준다.
    hard_voting_model_list.append(t1)

hard_voting_model = VotingClassifier(estimators=hard_voting_model_list, voting='hard')
hard_voting_model

VotingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                             ('LogisticRegression', LogisticRegression()),
                             ('SVC', SVC(probability=True)),
                             ('DecisionTree', DecisionTreeClassifier()),
                             ('RandomForest', RandomForestClassifier()),
                             ('AdaBoost', AdaBoostClassifier()),
                             ('GradientBoost', GradientBoostingClassifier()),
                             ('LGBM', LGBMClassifier(verbose=-1)),
                             ('XGBoost',
                              XGBC...
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))])

In [ ]:
# SoftVoting 구성
# 사용할 모델을 담을 리스트
soft_voting_model_list = []

# 사용하고자 하는 모델들의 이름으로 반복한다.
for m1 in new_model_name_list :
    # 모델을 생성한다.
    new_model = makeModel(m1)
    # 모델을 이름과 함께 튜플로 구성한다.
    t1 = (m1, new_model)
    # 모델 리스트에 담아준다.
    soft_voting_model_list.append(t1)

soft_voting_model = VotingClassifier(estimators=soft_voting_model_list, voting='soft')
soft_voting_model

VotingClassifier(estimators=[('KNN', KNeighborsClassifier()),
                             ('LogisticRegression', LogisticRegression()),
                             ('SVC', SVC(probability=True)),
                             ('DecisionTree', DecisionTreeClassifier()),
                             ('RandomForest', RandomForestClassifier()),
                             ('AdaBoost', AdaBoostClassifier()),
                             ('GradientBoost', GradientBoostingClassifier()),
                             ('LGBM', LGBMClassifier(verbose=-1)),
                             ('XGBoost',
                              XGBC...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')

In [ ]:
# Voting 객체들을 담아준다.
new_model_list.append(hard_voting_model)
new_model_list.append(soft_voting_model)

new_model_name_list.append('Voting(Hard)')
new_model_name_list.append('Voting(Soft)')

display(new_model_list)
display(new_model_name_list)

[KNeighborsClassifier(),
 LogisticRegression(),
 SVC(probability=True),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 AdaBoostClassifier(),
 GradientBoostingClassifier(),
 LGBMClassifier(verbose=-1),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               feature_weights=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, multi_strategy=None, n_estimators=None,
               n_jobs=None, num_parallel_tree=None, ...),
 VotingClassifier(

['KNN',
 'LogisticRegression',
 'SVC',
 'DecisionTree',
 'RandomForest',
 'AdaBoost',
 'GradientBoost',
 'LGBM',
 'XGBoost',
 'Voting(Hard)',
 'Voting(Soft)']

In [ ]:
# 교차 검증
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 사용할 모델의 수 만큼 반복한다.
for idx, m1 in enumerate(new_model_name_list) :
    print(f'{m1} 교차 검증 중')
    # idx 번째 학습 모델을 통해 교차 검증을 한다.
    r1 = cross_val_score(new_model_list[idx], train_X, train_y, scoring='f1', cv=kfold)
    # 교차 검증값을 리스트에 담는다.
    f1_score_list.append(r1.mean())

d1 = {
    '평균 f1 스코어' : f1_score_list
}

temp_df = pd.DataFrame(d1, index=new_model_name_list)
temp_df.sort_values(inplace=True, by='평균 f1 스코어', ascending=False)
temp_df['튜닝여부'] = 'X'
temp_df

KNN 교차 검증 중
LogisticRegression 교차 검증 중
SVC 교차 검증 중


### 하이퍼 파라미터 튜닝

In [ ]:
# 하이퍼 파라미터 튜닝을 위한 값들
# 튜닝할 필요가 없는 모델은 None을 셋팅한다.
# model_name_list 에 있는 모델들 중 Voting은 제외하고 순서대로 설정한다.
hyper_parameter_list = [
    # KNN
    {
        'n_neighbors' : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
    # LogisticRegression
    {
        'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
        'C' : [0.0001, 0.001, 0.01, 0.1, 0, 10, 100, 1000, 10000]
    },
    # SVC
    {
        'C' : [0.0001, 0.001, 0.01, 0.1, 0, 10, 100, 1000, 10000]
    },
    # DecisionTree
    {
        'max_depth' : [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    },
    # RandomForest
    {
        'n_estimators' : [50, 100, 150, 200, 250, 300]
    },
    # AdaBoost
    {
        'n_estimators' : [50, 100, 150, 200, 250, 300],
        'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
    },
    # GradientBoost
    {
        'n_estimators' : [50, 100, 150, 200, 250, 300],
        'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
    },
    # LGBM
    {
        'n_estimators' : [50, 100, 150, 200, 250, 300],
        'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
    },
    # XGBoost
    {
        'booster' : ['gbtree', 'gblinear'],
        'n_estimators' : [50, 100, 150, 200, 250, 300],
        'learning_rate' : [0.0001, 0.001, 0.01, 0.1, 1, 0, 10, 100, 1000, 10000]
    }
]

In [ ]:
new_tuning_model_list = []
new_tuning_model_name_list = []
new_tuning_hyper_parameter_list = []

# 사용하지 않을 모델은 제외한다.
for idx, m1 in enumerate(model_name_list) :
    # 사용하지 않을 모델이라면 다음 반복으로 넘어간다.
    if m1 in except_model_name_list :
        continue
    # Voting은 제외한다.
    if m1 in ['Voting(Hard)', 'Voting(Soft)'] :
        continue

    # 모델 객체를 생성한다.
    new_tuning_model = makeModel(m1)

    # 생성한 모델을 리스트에 담아준다.
    new_tuning_model_list.append(new_model)
    new_tuning_model_name_list.append(m1)
    new_tuning_hyper_parameter_list.append(hyper_parameter_list[idx])

display(new_tuning_model_list)
display(new_tuning_model_name_list)
display(new_tuning_hyper_parameter_list)

In [ ]:
# 최적의 하이퍼 파라미터 구성을 담을 리스트
best_hyper_parameter_list = []
# 하이퍼 파라미터 튜닝
kfold = KFold(n_splits=cv_count, shuffle=True, random_state=1)

# 사용할 모델의 수 만큼 반복한다.
for idx, m1 in enumerate(new_tuning_model_name_list) :
    print(f'{m1} 하이퍼 파라미터 튜닝 중')

    grid_clf = GridSearchCV(new_tuning_model_list[idx], param_grid=new_tuning_hyper_parameter_list[idx], scoring='f1', cv=kfold)
    grid_clf.fit(train_X, train_y)

    print(f'최적의 하이퍼 파라미터 : {grid_clf.best_params_}')
    print(f'최적의 모델 평균 성능 : {grid_clf.best_score_}')
    print()

    # 최적의 모델 평균 성능을 담아준다.
    f1_score_tuning_list.append(grid_clf.best_score_)
    # 하이퍼 파라미터 구성을 담아준다.
    best_hyper_parameter_list.append(grid_clf.best_params_)

d2 = {
    '평균 f1 스코어' : f1_score_tuning_list
}

temp_df = pd.DataFrame(d2, index=new_tuning_model_name_list)
temp_df.sort_values(inplace=True, by='평균 f1 스코어', ascending=False)
temp_df['튜닝여부'] = 'O'
temp_df

In [ ]:
# 여기에서는 하이퍼 파라미터를 보고 모델 객체들을 생성하여
# Voting 을 구성하여 하이퍼 파라미터를 튜닝한다.
# 실행시 상단의 전체 실행으로

# 하이퍼 파라미터 출력
display(new_tuning_model_name_list)
display(best_hyper_parameter_list)

In [ ]:
# 위에서 출력된 결과를 보고 하이퍼 파라미터를 설정한 학습모델을 만들어준다
# HardVoting 구성
#KNN
model1 = ("KNN Tuning", KNeighborsClassifier(n_neighbors=1))

# 사용할 모델을 담을 리스트
hard_voting_tuning_model_list = [
    model1
]

hard_voting_tuning_model = VotingClassifier(estimators=hard_voting_tuning_model_list, voting='hard')
hard_voting_tuning_model

In [ ]:
# 위에서 출력된 결과를 보고 하이퍼 파라미터를 설정한 학습모델을 만들어준다
# SoftVoting 구성
#KNN
model1 = ("KNN Tuning", KNeighborsClassifier(n_neighbors=1))

# 사용할 모델을 담을 리스트
soft_voting_tuning_model_list = [
    model1
]

soft_voting_tuning_model = VotingClassifier(estimators=soft_voting_tuning_model_list, voting='soft')
soft_voting_tuning_model

In [ ]:
# 튜닝된 모델로 구성된 보팅을 모델을 교차검증하여 결과를 담아준다.
r1 = cross_val_score(hard_voting_tuning_model, train_X, train_y, scoring='f1', cv=kfold)
r2 = cross_val_score(soft_voting_tuning_model, train_X, train_y, scoring='f1', cv=kfold)

f1_score_tuning_list.append(r1.mean())
f1_score_tuning_list.append(r2.mean())

new_tuning_model_name_list.append('Voting(Hard)')
new_tuning_model_name_list.append('Voting(Soft)')

display(f1_score_tuning_list)
display(new_tuning_model_name_list)

In [ ]:
# 최종 결과 확인을 위해서 데이터 프레임을 생성한다.
d1 = {
    'f1 score' : f1_score_list
}
temp_df1 = pd.DataFrame(d1, index=new_model_name_list)
temp_df1['tuning'] = 'X'

d2 = {
    'f1 score' : f1_score_tuning_list
}
temp_df2 = pd.DataFrame(d2, index=new_tuning_model_name_list)
temp_df2['tuning'] = 'O'

# 두 데이터 프레임을 하나로 합친다.
result_df = pd.concat([temp_df1, temp_df2])
# f1 score를 기준으로 내림차순 정렬한다.
result_df.sort_values(by='f1 score', inplace=True, ascending=False)
result_df

### 학습 모델 저장

In [ ]:
# 위의 결과를 보고 최적이라 판단되는 학습 모델 객체를 생성한다.
best_model = XGBClassifier()
# 전체 데이터를 학습한다.
best_model.fit(train_X, train_y)

In [ ]:
# 저장한다.
# best_model_path에 있는 폴더를 꼭 먼저 만들어주요요
with open(best_model_path, mode='wb') as fp :
    pickle.dump(best_model, fp)
    # 이후에는 사용한 인코더나 스케일러를 모두 저장한다.
    pickle.dump(encoder1, fp)
    pickle.dump(scaler1, fp)

print('저장완료')

In [ ]:
# 확인을 위해 예측을 수행해본다.
# 저장된 학습 모델 등을 불러온다.
with open(best_model_path, mode = 'rb') as fp :
    test_model = pickle.load(fp)
    test_encoder = pickle.load(fp)
    test_scaler = pickle.load(fp)

print(test_model)
print(test_encoder)
print(test_scaler)